# Code 6
- Preprocessing Improvement
- Decision Tree

## 1/ Import Libraries

In [ ]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [ ]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [ ]:
#CodeSection3
train = pd.read_csv('/mntDrive/MyDrive/__DataScience/2 Cross Sell Prediction (Cohort)/input/train.csv')
test = pd.read_csv('/mntDrive/MyDrive/__DataScience/2 Cross Sell Prediction (Cohort)/input/test.csv')

## 3/ Combine both Datasets (train, test)

In [ ]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [ ]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

id                           0
Gender                       0
Age                          0
Driving_License              0
Region_Code                  0
Previously_Insured           0
Vehicle_Age                  0
Vehicle_Damage               0
Annual_Premium               0
Policy_Sales_Channel         0
Vintage                      0
Response                127037
dtype: int64

## 4/ Preprocessing Improvement

### 4.1/ Identify Numerical and Categorical Features

In [ ]:
#CodeSection6
# Identify all Numerical and Categorical features
numerical_features = [ 'Annual_Premium', 'Region_Code','Policy_Sales_Channel','Vintage']
categorical_features = ['Gender','Age', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']

### 4.2/ Outlier Strategy

In [ ]:
#CodeSection7

# Write a loop to do the same
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  train[num_var] = train[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)
  test[num_var] = test[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)

### 4.3/ Import Libraries
- Missing Value Imputation - SimpleImputer
- Preprocessing - StandardScaler, OrdinalEncoder
- Pipeline - make_pipeline, make_column_transformer
- Model - Decision Tree

In [ ]:
#CodeSection8

# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Import StandardScaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Make and Compose Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Import Decision tree
from sklearn.tree import DecisionTreeClassifier

### 4.4/ Build Pipeline

In [ ]:
#CodeSection9
# Create Preprocessor Pipeline
preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy='median'), StandardScaler()),numerical_features),
    (make_pipeline(SimpleImputer(strategy='most_frequent'),OrdinalEncoder()),categorical_features)
)


### 4.5/ Divide Data into X and y

In [ ]:
#CodeSection10
X = train.drop(['id','Response'], axis =  1)
y = train['Response']

## 5/ Build Model and Fit

In [ ]:
#CodeSection11
# Create Model Pipeline and Initiate Model
model = make_pipeline(preprocessor,DecisionTreeClassifier(max_depth=5))

In [ ]:
#CodeSection12 
# Fit Model
model.fit(X,y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

## 6/ Check Accuracy of Model on Train Data

In [ ]:
#CodeSection13
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [ ]:
#CodeSection14
# Import metrics library
from sklearn.metrics import accuracy_score

In [ ]:
#CodeSection15
# get Actual "y" variables (use "y_true" as variable)
y_true = train['Response']

In [ ]:
#CodeSection16
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

 Train Accuracy : 87.7%


## 7/ Predict and Submission

### Predict on "Test Data"

In [ ]:
X_test.columns

Index(['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage'],
      dtype='object')

In [ ]:
#CodeSection17
# Get all the X Variables from the Test Dataset
X_test = test.drop(['id'], axis =  1)

In [ ]:
#CodeSection18
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [ ]:
#CodeSection19
submission = pd.DataFrame({
    'id' : test['id'],
    'Response' : X_test_prep
})

### Export Submission File

In [ ]:
#CodeSection20
submission.to_csv('/mntDrive/MyDrive/__DataScience/2 Cross Sell Prediction (Cohort)/output/O6_DT_Pipeline.csv', index = False)

In [ ]:
# LB Accuracy : 0.58